<h1>Import all libraries and reading explored data into Dataframe</h1>

In [1]:
import re, io, gensim, datetime, time, nltk, random, pickle
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
#General libraries needed
import numpy as np
import pandas as pd

#Libraries for data pre-processing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import preprocessing

#For Decision Tree implementation
from scipy.stats import entropy
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

#For Bagging implementation
from sklearn.ensemble import BaggingClassifier

#For AdaBoost implementation
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

#For Random Forest implementation
from sklearn.ensemble import RandomForestClassifier

#For Logistic Regression
from sklearn.linear_model import LogisticRegression

#For SVClinear
from sklearn.svm import LinearSVC

string = 'true:'
goldtruth = [string+str(i) for i in range(0, 58)]

string = 'pred:'
prediction = [string+str(i) for i in range(0, 58)]

def printModelAccuracy(y_test, y_pred,file_name):
    # Find the confusion matrix of the result
    cm = pd.DataFrame(confusion_matrix(y_test, y_pred, labels=list(range(0, 58))), \
        index=goldtruth, 
        columns=prediction)
    print("Confusion Matrix:")
    print(cm)
    cm.to_csv(file_name)
    
    # Find the accuracy and F1 score of the result
    asr = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("Accuracy:", asr)
    print("F1:", f1)

C:\Users\Admin\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
X_test = pd.read_pickle("X_test_best_choice_6")
X_train = pd.read_pickle("X_train_best_choice_6")
y_test = pd.read_pickle("y_test_best_choice_6")
y_train = pd.read_pickle("y_train_best_choice_6")

<h1>Naive bayes</h1>

In [3]:
naivebayes = MultinomialNB()
#Fit the training feature Xs and training label Ys
naivebayes.fit(X_train, y_train.values.ravel())

#Use the trained model to predict the test data
y_pred = naivebayes.predict(X_test)

# Find the confusion matrix, the accuracy, and F1 score of the result
printModelAccuracy(y_test, y_pred,'cm_naive_bayes.csv')


Confusion Matrix:
         pred:0  pred:1  pred:2  pred:3  pred:4  pred:5  pred:6  pred:7  \
true:0       16       6      10       5       7       3       1       3   
true:1        0     272       4      43      28      66       1       6   
true:2        0       2     116       7      14       2       0       3   
true:3        0       8       3     944      34      91       0       3   
true:4        0       1       0      56     428      94       0       8   
true:5        0       1       1      25     104     611       0       4   
true:6        0       0       0       3       0       0       0      21   
true:7        0       0       0       4      20      15       0     109   
true:8        0       0       0       6       2       4       0      16   
true:9        0       0       0       8      25      24       0       3   
true:10       0       0       0       1       8       0       0       0   
true:11       0       0       0      11       1       4       0      12   
true:12

<h1>Logistic Regression</h1>

In [4]:
log_reg = LogisticRegression()
#Fit the training feature Xs and training label Ys
log_reg.fit(X_train, y_train.values.ravel())

#Use the trained model to predict the test data
y_pred = log_reg.predict(X_test)

# Find the confusion matrix, the accuracy, and F1 score of the result
printModelAccuracy(y_test, y_pred,'cm_log_reg.csv')

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Confusion Matrix:
         pred:0  pred:1  pred:2  pred:3  pred:4  pred:5  pred:6  pred:7  \
true:0       35      15       3       1       1       0       0       1   
true:1        0     376       4      22      17       2       0       4   
true:2        0       1     130       5      13       0       0       0   
true:3        0       5       6     976      28      58       1       0   
true:4        0       0       3      42     468      61       2       4   
true:5        0       0       1      30      93     613       0       3   
true:6        0       0       0       4       1       0      19       7   
true:7        0       0       0       3      15       5       3     122   
true:8        0       0       0       1       2       4       5       2   
true:9        0       0       0       7      14      16       0       1   
true:10       0       0       0       1       2       0       0       0   
true:11       0       0       0       4       1       3       3       6   
true:12

<h1>AdaBoost (with Decision Tree)</h1>

In [5]:
#Create the AdaBoost classifier. Default base classifiers is Decision Tree. 
# - n_estimator is the number of base classifiers (i.e. weak learners)
# - learning_rate controls the weight adjustments of each base classifiers. Default is 1
adaboostTree = AdaBoostClassifier(n_estimators=50,learning_rate=1)

#Fit the training feature Xs and training label Ys
adaboostTree.fit(X_train, y_train.values.ravel())

#Use the trained model to predict the test data
y_pred = adaboostTree.predict(X_test)

# Find the confusion matrix, the accuracy, and F1 score of the result
printModelAccuracy(y_test, y_pred,'cm_ada_decision_tree.csv')

Confusion Matrix:
         pred:0  pred:1  pred:2  pred:3  pred:4  pred:5  pred:6  pred:7  \
true:0       58       0       0       0       0       0       0       0   
true:1        0       0       0     426       0       0       0       0   
true:2        0       0       0     156       0       0       0       0   
true:3        0       0       0    1093       0       0       0       0   
true:4        0       0       0       0     619       0       0       0   
true:5        0       0       0       0       0     747       0       0   
true:6        0       0       0       0       0       0       0       0   
true:7        0       0       0       0       0       0       0       0   
true:8        0       0       0       0       0       0       0       0   
true:9        0       0       0       0       0       0       0       0   
true:10       0       0       0       0       0       0       0       0   
true:11       0       0       0       0       0       0       0       0   
true:12

<h1>AdaBoost (with Naive Bayes)</h1>

In [6]:
nb = MultinomialNB()

adaboostnaivebayes = AdaBoostClassifier(n_estimators=50,learning_rate=1, base_estimator=nb)
#model = BaggingClassifier(n_estimators=50, base_estimator=knn)

#Fit the training feature Xs and training label Ys
adaboostnaivebayes.fit(X_train, y_train.values.ravel())

#Use the trained model to predict the test data
y_pred = adaboostnaivebayes.predict(X_test)

# Find the confusion matrix, the accuracy, and F1 score of the result
printModelAccuracy(y_test, y_pred,'cm_ada_boost_naive.csv')

Confusion Matrix:
         pred:0  pred:1  pred:2  pred:3  pred:4  pred:5  pred:6  pred:7  \
true:0        0       0       0       0       0       0       0       0   
true:1        0       0       0       0       0       0       0       0   
true:2        0       0       0       0       0       0       0       0   
true:3        0       0       0       0       0       0       0       0   
true:4        0       0       0       0       0       0       0       0   
true:5        0       0       0       0       0       0       0       0   
true:6        0       0       0       0       0       0       0       0   
true:7        0       0       0       0       0       0       0       0   
true:8        0       0       0       0       0       0       0       0   
true:9        0       0       0       0       0       0       0       0   
true:10       0       0       0       0       0       0       0       0   
true:11       0       0       0       0       0       0       0       0   
true:12

<h1>Random Forest</h1>

In [7]:
#Instantiate model with 500 decision trees
rf = RandomForestClassifier(n_estimators = 500, random_state = 0, max_depth = 2)

#Fit the training feature Xs and training label Ys
rf.fit(X_train, y_train.values.ravel())

#Use the trained model to predict the test data
y_pred = rf.predict(X_test)

# Find the confusion matrix, the accuracy, and F1 score of the result
printModelAccuracy(y_test, y_pred,'cm_random_forest.csv')

Confusion Matrix:
         pred:0  pred:1  pred:2  pred:3  pred:4  pred:5  pred:6  pred:7  \
true:0        0       0       0      58       0       0       0       0   
true:1        0       0       0     426       0       0       0       0   
true:2        0       0       0     156       0       0       0       0   
true:3        0       0       0    1093       0       0       0       0   
true:4        0       0       0     617       0       2       0       0   
true:5        0       0       0     689       0      58       0       0   
true:6        0       0       0      40       0       0       0       0   
true:7        0       0       0     162       0       0       0       0   
true:8        0       0       0      72       0       0       0       0   
true:9        0       0       0     105       0       0       0       0   
true:10       0       0       0      19       0       0       0       0   
true:11       0       0       0      59       0       0       0       0   
true:12

<h1>LinearSVC</h1>

In [8]:
clf = LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=1e-05, verbose=0)

clf.fit(X_train, y_train.values.ravel())

#Use the trained model to predict the test data
y_pred = clf.predict(X_test)

# Find the confusion matrix, the accuracy, and F1 score of the result
printModelAccuracy(y_test, y_pred,'cm_linear_svc.csv')


C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Confusion Matrix:
         pred:0  pred:1  pred:2  pred:3  pred:4  pred:5  pred:6  pred:7  \
true:0       36      10       8       1       1       0       0       0   
true:1        1     381       5      22      10       1       0       3   
true:2        0       3     130       5      11       0       0       2   
true:3        0       5       9     970      30      60       2       0   
true:4        0       0       5      52     432      72       2       8   
true:5        0       0       1      37      94     599       0       4   
true:6        0       0       0       2       0       0      20       8   
true:7        0       0       0       3      13       7       4     117   
true:8        0       0       0       0       2       3       7       1   
true:9        0       0       0       3      17      13       0       1   
true:10       0       0       0       0       2       0       0       0   
true:11       0       0       0       2       0       3       3       6   
true:12

<h1>Bagging</h1>

<h1>Save the best model and load the best model</h1>

In [10]:
#save classifier
save_classifier = open("logregmodel.pickle","wb") #binary write
pickle.dump(log_reg, save_classifier)
save_classifier.close()

#Load the save classifier 
classifier_saved = open("logregmodel.pickle", "rb") #binary read
classifier_load = pickle.load(classifier_saved)
classifier_saved.close()
